# INTRODUCCION

En este cuaderno cargamos los datos que procesamos anteriormente:
*   Quitamos los duplicados
*   quitamos las columnas innecesarias que no aportaban valor, ID, nombre, apellidos, telefono
*   hemos dejado el dataset con los NaN sin tocar

Veamos otra opcion a realizar con estos NaN que no vimos en el cuaderno anterior

###Imputar usando *Machine Learning*

Es el método **más robusto y más recomendado**.

Consiste en construir un modelo de *Machine Learning* que tome las variables que están completas y aprenda a predecir la variable incompleta :


**Ventajas**
- Se preserva la cantidad de datos
- No se generan sesgos (siempre y cuando el modelo pueda ser construido correctamente)

**Desventajas**
- Se requieren suficientes datos para entrenar el modelo: no debe haber demasiados datos faltantes ni muy pocos datos de entrenamiento
- Dependiendo del set de datos no siempre resulta sencillo construir un modelo que genere predicciones adecuadas

**Procedimiento paso a paso**
1. Crear el set de entrenamiento: registros que contienen datos completos
2. Crear el set de prueba: registros que contienen datos incompletos
3. Escoger y entrenar el modelo de *Machine Learning* con el set de entrenamiento
4. Predecir datos faltantes con el modelo entrenado y con el set de prueba
5. Incorporar los datos predichos en el dataset

Veamos en este ejemplo cómo implementar cada paso:

In [7]:
# Importamos las librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.io as pio
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np
from yellowbrick.cluster import KElbowVisualizer
import warnings
warnings.filterwarnings("ignore")
py.offline.init_notebook_mode(connected = True)
pio.renderers.default='browser'

In [3]:
# Traer el CSV y ver la informacion relevante

df_eda = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/EDA/Datos/EDA_01.csv', sep=",", na_values=' ?')
# despues del separador podemos añadir header= None, para decir que este dataframe no tiene cabecera
# tamien podemos añadir na_values=' ?' para indicar que si encuetra ese caracter lo tome como NaN

# la informacion relevante para un inicio y primera toma de contacto

print('Dimensiones del df:', df_eda.shape)
print()
# Mostrar el encabezado (primeras 3 filas)
print("Encabezado del dataset:")
display(df_eda.head(3))
print()
# Mostrar 3 filas al azar
print("\n3 filas al azar del dataset:")
display(df_eda.sample(3))
print()
# Mostrar la cola (últimas 3 filas)
print("\nCola del dataset:")
display(df_eda.tail(3))
print()
print("con info")
df_eda.info()

Dimensiones del df: (54, 6)

Encabezado del dataset:


,edad,sexo,estado salud,ingresos,deporte,educacion
0,9,hombre,Malo,Bajo,Sí,Secundaria
1,22,mujer,Normal,Medio,No,Primaria
2,25,hombre,Excelente,Alto,Sí,Universitaria




3 filas al azar del dataset:


,edad,sexo,estado salud,ingresos,deporte,educacion
15,58,mujer,Malo,Medio,No,NaN
31,71,mujer,Otro,Alto,Sí,Secundaria
20,40,NaN,Malo,Alto,Sí,Primaria




Cola del dataset:


,edad,sexo,estado salud,ingresos,deporte,educacion
51,16,hombre,Malo,Bajo,No,Secundaria
52,60,mujer,Malo,Alto,Sí,NaN
53,58,mujer,Malo,Medio,No,Secundaria



con info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   edad          54 non-null     int64 
 1   sexo          50 non-null     object
 2   estado salud  47 non-null     object
 3   ingresos      51 non-null     object
 4   deporte       51 non-null     object
 5   educacion     50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB


Como podemos observar tenemos NaN esparcidos por todo el dataframe

En este caso son pocos los datos como para poder entrenar a nuestro modelo, pero debemos tener en cuenta que en la vida real esta base de datos suele ser mas grande.

Para poder manejar nos mejor, vamos a comenzar prediciendo la columna sexo, por ese motivo la vamos a mover al principio, a la posicion cero

In [4]:
# Método 1: Usando pop() e insert()
columna = df_eda.pop('sexo')  # Extrae la columna del DataFrame
df_eda.insert(0, 'sexo', columna)  # La inserta en la posición 0

# Método 2: Reordenando las columnas
# df_eda = df_eda[['sexo'] + [col for col in df_eda.columns if col != 'sexo']]

df_eda.head()


,sexo,edad,estado salud,ingresos,deporte,educacion
0,hombre,9,Malo,Bajo,Sí,Secundaria
1,mujer,22,Normal,Medio,No,Primaria
2,hombre,25,Excelente,Alto,Sí,Universitaria
3,mujer,27,Normal,Bajo,No,Postgrado
4,NaN,30,NaN,Medio,Sí,Secundaria


In [5]:
df_num = df_eda.copy()
df_eda.info()
df_eda.head()

for col in df_num.columns:
    print(f"\nValores únicos en {col}:")
    print(df_num[col].value_counts(dropna=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sexo          50 non-null     object
 1   edad          54 non-null     int64 
 2   estado salud  47 non-null     object
 3   ingresos      51 non-null     object
 4   deporte       51 non-null     object
 5   educacion     50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB

Valores únicos en sexo:
sexo
hombre    25
mujer     25
NaN        4
Name: count, dtype: int64

Valores únicos en edad:
edad
25    3
58    3
51    3
9     2
40    2
60    2
22    2
64    2
65    2
35    2
30    2
27    2
43    1
15    1
44    1
33    1
80    1
49    1
66    1
41    1
19    1
55    1
76    1
23    1
38    1
47    1
46    1
29    1
71    1
52    1
18    1
32    1
63    1
61    1
56    1
54    1
37    1
48    1
16    1
Name: count, dtype: int64

Valores únicos en estado salud:
estado salud
Mal

El tercer paso es escoger y entrenar el modelo de Machine Learning con el set de entrenamiento.

Para este ejemplo podemos usar un sencillo modelo de Regresión Logística.

Pero antes de entrenarlo debemos pre-procesar los datos, pues las categorías a predecir (`masculino`, `femenino`) no pueden estar en formato de texto sino que deben estar en formato numérico (0 ó 1).

Para hacer esta conversión podemos usar `LabelEncoder` de *Scikit Learn*:

In [8]:
# Crear una copia de df_eda para no modificar el original
df_num = df_eda.copy()

# Crear un diccionario para almacenar los encoders
encoders = {}

# Creación de lista con variables categóricas
catCols = ['sexo', 'estado salud', 'ingresos', 'deporte', 'educacion']

# Iterar sobre cada columna categórica
for col in catCols:
    # Crear un LabelEncoder
    le = LabelEncoder()

    # Obtener los valores no nulos
    non_null = df_num[col].dropna()

    # Ajustar el LabelEncoder y transformar los valores no nulos
    df_num.loc[non_null.index, col] = le.fit_transform(non_null)

    # Guardar el encoder en el diccionario
    encoders[col] = le

# Imprimir las primeras filas del DataFrame resultante
print(df_num[catCols].head())

# Imprimir los valores únicos de cada columna categórica
for col in catCols:
    print(f"\nValores únicos en {col}:")
    print(df_num[col].value_counts(dropna=False))

# Imprimir las clases de cada encoder
for col, encoder in encoders.items():
    print(f"\nClases para {col}:")
    print(encoder.classes_)



  sexo estado salud ingresos deporte educacion
0    0            1        1       1         2
1    1            2        2       0         1
2    0            0        0       1         3
3    1            2        1       0         0
4  NaN          NaN        2       1         2

Valores únicos en sexo:
sexo
0      25
1      25
NaN     4
Name: count, dtype: int64

Valores únicos en estado salud:
estado salud
1      25
0      10
2       9
NaN     7
3       3
Name: count, dtype: int64

Valores únicos en ingresos:
ingresos
1      19
2      18
0      14
NaN     3
Name: count, dtype: int64

Valores únicos en deporte:
deporte
1      27
0      24
NaN     3
Name: count, dtype: int64

Valores únicos en educacion:
educacion
2      15
3      13
1      12
0      10
NaN     4
Name: count, dtype: int64

Clases para sexo:
['hombre' 'mujer']

Clases para estado salud:
['Excelente' 'Malo' 'Normal' 'Otro']

Clases para ingresos:
['Alto' 'Bajo' 'Medio']

Clases para deporte:
['No' 'Sí']

Clases para ed

In [9]:
# Crear el set de entrenamiento

# Extraer las filas que contienen datos completos
XY = df_num.dropna().to_numpy()
XY.shape

for col in df_num.columns:
    print(f"\nValores únicos en {col}:")
    print(df_num[col].value_counts(dropna=False))


Valores únicos en sexo:
sexo
0      25
1      25
NaN     4
Name: count, dtype: int64

Valores únicos en edad:
edad
25    3
58    3
51    3
9     2
40    2
60    2
22    2
64    2
65    2
35    2
30    2
27    2
43    1
15    1
44    1
33    1
80    1
49    1
66    1
41    1
19    1
55    1
76    1
23    1
38    1
47    1
46    1
29    1
71    1
52    1
18    1
32    1
63    1
61    1
56    1
54    1
37    1
48    1
16    1
Name: count, dtype: int64

Valores únicos en estado salud:
estado salud
1      25
0      10
2       9
NaN     7
3       3
Name: count, dtype: int64

Valores únicos en ingresos:
ingresos
1      19
2      18
0      14
NaN     3
Name: count, dtype: int64

Valores únicos en deporte:
deporte
1      27
0      24
NaN     3
Name: count, dtype: int64

Valores únicos en educacion:
educacion
2      15
3      13
1      12
0      10
NaN     4
Name: count, dtype: int64


###Preparación de datos para entrenamiento
Este código está preparando los datos para entrenar un modelo de machine learning. Está dividiendo los datos en dos partes principales: las características (features) que usaremos para predecir, y la etiqueta (label) que queremos predecir.


*    Esta línea está creando x_train, que contiene las características para el entrenamiento.
*   XY parece ser una matriz o array que contiene todos nuestros datos.
*    [:,1:6] es una forma de seleccionar columnas en Python:
 *       : significa "todas las filas"
 *        1:6 significa "desde la columna 1 hasta la 5" (en Python, el último número no se incluye)   Así, estamos seleccionando las columnas 1, 2, 3, 4 y 5 de XY, que representan edad, estado de salud, ingresos, deporte y educación.


In [10]:
# Set de entrenamiento
# x_train: columnas edad, estado salud, ingresos, deporte y educacion
# y_train: columna  ("sexo")
x_train = XY[:,1:6]
y_train = XY[:,0]

y_train = XY[:,0].astype(int)

print(x_train)
print(y_train)

[[9 1 1 1 2]
 [22 2 2 0 1]
 [25 0 0 1 3]
 [27 2 1 0 0]
 [35 2 0 1 3]
 [37 0 2 0 2]
 [40 2 1 1 0]
 [48 1 1 0 2]
 [54 1 1 0 0]
 [56 2 0 1 3]
 [61 1 1 1 1]
 [65 1 1 0 2]
 [51 2 2 0 0]
 [9 1 1 1 3]
 [22 0 2 0 2]
 [27 0 1 0 3]
 [51 0 2 1 1]
 [64 1 2 1 2]
 [35 1 0 0 0]
 [52 1 2 0 3]
 [71 3 0 1 2]
 [29 1 1 0 1]
 [47 3 2 1 0]
 [38 0 1 1 2]
 [23 1 2 0 1]
 [60 1 0 1 0]
 [55 2 2 1 2]
 [41 1 1 1 3]
 [30 1 2 0 2]
 [25 0 1 0 1]
 [49 1 2 1 3]
 [33 1 1 1 1]
 [44 2 2 0 0]
 [58 3 0 1 3]
 [16 1 1 0 2]
 [58 1 2 0 2]]
[0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 0 1]


El siguiente código está creando un conjunto de datos de prueba (x_test) a partir de un DataFrame llamado df_num. El objetivo es seleccionar filas donde el 'sexo' es desconocido (NaN) pero todas las otras columnas relevantes tienen datos.

De esta manera nos asegurams que todos los datos de entrenamiento tienen datos.

In [11]:
# Crear el set de prueba: filas con datos incompletos
# y columnas edad, estado salud, ingresos, deporte y educacion

# Seleccionar filas donde 'sexo' es NaN
filas_sexo_nan = df_num['sexo'].isna()

# Excluir filas que tienen NaN en otras columnas
otras_columnas = ['edad', 'estado salud', 'ingresos', 'deporte', 'educacion']
filas_sin_nan_otras = df_num[otras_columnas].notna().all(axis=1)

# Combinar las condiciones
filas_seleccionadas = filas_sexo_nan & filas_sin_nan_otras

# Crear x_test
x_test = df_num[otras_columnas].loc[filas_seleccionadas].to_numpy()

print("Forma de x_test:", x_test.shape)
print("\nPrimeras filas de x_test:")
print(x_test[:5])  # Muestra las primeras 5 filas (o menos si hay menos)


Forma de x_test: (3, 5)

Primeras filas de x_test:
[[32 1 1 0 1]
 [64 0 2 1 3]
 [40 1 0 1 1]]


La Regresión Logística es un algoritmo de clasificación que se usa cuando queremos predecir una variable categórica (como en este caso, el sexo). A pesar de su nombre, se usa para clasificación, no para regresión. Algunas características importantes:

*    Es bueno para problemas de clasificación binaria (dos clases).
*    Produce resultados probabilísticos (la probabilidad de pertenecer a cada clase).
*    Es relativamente simple y rápido de entrenar.
*    Funciona bien con conjuntos de datos linealmente separables.


In [12]:
# Importar el módulo
from sklearn.linear_model import LogisticRegression

# Crear instancia del modelo
lr = LogisticRegression()

# Entrenarlo con "fit" y con los datos de entrenamiento
lr.fit(x_train,y_train)

LogisticRegression()

In [13]:
# Predicciones sobre el set de prueba
preds = lr.predict(x_test)
print("Predicciones numéricas:", preds)

# Transformaciones inversas
# Primero, necesitamos definir las etiquetas originales de 'sexo'
etiquetas_sexo = df_eda['sexo'].unique()

# Creamos un diccionario para mapear los valores numéricos a las etiquetas
mapeo_sexo = {i: etiqueta for i, etiqueta in enumerate(etiquetas_sexo)}

# Ahora hacemos la transformación inversa
cats = [mapeo_sexo[pred] for pred in preds]
print("Predicciones en categorías:", cats)


Predicciones numéricas: [0 1 0]
Predicciones en categorías: ['hombre', 'mujer', 'hombre']


In [ ]:
assert (len(etiquetas_sexo) == len(np.unique(preds)), "El número de categorías predichas no coincide con las etiquetas originales")


In [ ]:
df_num.head()

,sexo,edad,estado salud,ingresos,deporte,educacion
0,0,9,1,1,1,2
1,1,22,2,2,0,1
2,0,25,0,0,1,3
3,1,27,2,1,0,0
4,NaN,30,NaN,2,1,2


In [14]:
# Crear el DataFrame final y descodificar todas las columnas categóricas

# Crear un DataFrame con los datos de x_test
df_test = pd.DataFrame(x_test, columns=['edad', 'estado salud', 'ingresos', 'deporte', 'educacion'])

# Añadir la columna de predicciones
df_test['sexo_predicho'] = cats

# Verificar que el número de predicciones coincide con el número de filas en x_test
assert len(cats) == len(df_test), "El número de predicciones no coincide con el número de filas en x_test"

# Obtener los índices originales de las filas de prueba
indices_test = df_num[filas_seleccionadas].index

# Asignar los índices originales a df_test
df_test.index = indices_test

# Crear el DataFrame final
df_ml = df_test.copy()
df_ml['sexo'] = cats

# Descodificar las columnas categóricas
columnas_a_descodificar = ['estado salud', 'ingresos', 'deporte', 'educacion']

for col in columnas_a_descodificar:
    if col in encoders:
        df_ml[col] = encoders[col].inverse_transform(df_ml[col].astype(int))

# Mostrar un resumen de los cambios
print("Distribución de valores en la columna 'sexo':")
print(df_ml['sexo'].value_counts(dropna=False))

print("\nDistribución de valores en la columna 'sexo_predicho':")
print(df_ml['sexo_predicho'].value_counts(dropna=False))

# Mostrar las primeras filas del DataFrame actualizado
print("\nPrimeras filas del DataFrame actualizado:")
print(df_ml)

# Mostrar información detallada de cada columna
for col in df_ml.columns:
    print(f"\nValores únicos en {col}:")
    print(df_ml[col].value_counts(dropna=False))


Distribución de valores en la columna 'sexo':
sexo
hombre    2
mujer     1
Name: count, dtype: int64

Distribución de valores en la columna 'sexo_predicho':
sexo_predicho
hombre    2
mujer     1
Name: count, dtype: int64

Primeras filas del DataFrame actualizado:
   edad estado salud ingresos deporte      educacion sexo_predicho    sexo
5    32         Malo     Bajo      No       Primaria        hombre  hombre
18   64    Excelente    Medio      Sí  Universitaria         mujer   mujer
20   40         Malo     Alto      Sí       Primaria        hombre  hombre

Valores únicos en edad:
edad
32    1
64    1
40    1
Name: count, dtype: int64

Valores únicos en estado salud:
estado salud
Malo         2
Excelente    1
Name: count, dtype: int64

Valores únicos en ingresos:
ingresos
Bajo     1
Medio    1
Alto     1
Name: count, dtype: int64

Valores únicos en deporte:
deporte
Sí    2
No    1
Name: count, dtype: int64

Valores únicos en educacion:
educacion
Primaria         2
Universitaria    1
N

In [15]:
# Insertar las predicciones en df_num
df_num.loc[filas_seleccionadas, 'sexo'] = cats

# Eliminar la columna sexo_predicho si existe
if 'sexo_predicho' in df_num.columns:
    df_num = df_num.drop('sexo_predicho', axis=1)

# Mostrar las primeras y últimas filas del DataFrame actualizado
print("Primeras filas del DataFrame df_num actualizado:")
print(df_num.head(10))

print("\nÚltimas filas del DataFrame df_num actualizado:")
print(df_num.tail(10))

# Mostrar la distribución de valores en la columna 'sexo'
print("\nDistribución de valores en la columna 'sexo':")
print(df_num['sexo'].value_counts(dropna=False))

# Verificar cuántos valores NaN quedan en la columna 'sexo'
print(f"\nNúmero de valores NaN en 'sexo': {df_num['sexo'].isna().sum()}")


Primeras filas del DataFrame df_num actualizado:
     sexo  edad estado salud ingresos deporte educacion
0       0     9            1        1       1         2
1       1    22            2        2       0         1
2       0    25            0        0       1         3
3       1    27            2        1       0         0
4     NaN    30          NaN        2       1         2
5  hombre    32            1        1       0         1
6       0    35            2        0       1         3
7       1    37            0        2       0         2
8       0    40            2        1       1         0
9       1    43            1        2     NaN         1

Últimas filas del DataFrame df_num actualizado:
   sexo  edad estado salud ingresos deporte educacion
44    1    66            2      NaN       1         0
45    0    25            0        1       0         1
46    1    49            1        2       1         3
47    0    80          NaN        0       0         2
48    1    33   

In [23]:
# Insertar las predicciones en df_eda
df_eda.loc[filas_seleccionadas, 'sexo'] = cats

# Eliminar la columna sexo_predicho si existe
if 'sexo_predicho' in df_num.columns:
    df_eda = df_eda.drop('sexo_predicho', axis=1)

# Mostrar las primeras y últimas filas del DataFrame actualizado
print("Primeras filas del DataFrame df_eda actualizado:")
print(df_eda.head(10))

print("\nÚltimas filas del DataFrame df_eda actualizado:")
print(df_eda.tail(10))

# Mostrar la distribución de valores en la columna 'sexo'
print("\nDistribución de valores en la columna 'sexo':")
print(df_eda['sexo'].value_counts(dropna=False))

# Verificar cuántos valores NaN quedan en la columna 'sexo'
print(f"\nNúmero de valores NaN en 'sexo': {df_eda['sexo'].isna().sum()}")


Primeras filas del DataFrame df_eda actualizado:
     sexo  edad estado salud ingresos deporte      educacion
0  hombre     9         Malo     Bajo      Sí     Secundaria
1   mujer    22       Normal    Medio      No       Primaria
2  hombre    25    Excelente     Alto      Sí  Universitaria
3   mujer    27       Normal     Bajo      No      Postgrado
4     NaN    30          NaN    Medio      Sí     Secundaria
5  hombre    32         Malo     Bajo      No       Primaria
6  hombre    35       Normal     Alto      Sí  Universitaria
7   mujer    37    Excelente    Medio      No     Secundaria
8  hombre    40       Normal     Bajo      Sí      Postgrado
9   mujer    43         Malo    Medio     NaN       Primaria

Últimas filas del DataFrame df_eda actualizado:
      sexo  edad estado salud ingresos deporte      educacion
44   mujer    66       Normal      NaN      Sí      Postgrado
45  hombre    25    Excelente     Bajo      No       Primaria
46   mujer    49         Malo    Medio      S

In [17]:
for col, encoder in encoders.items():
    print(f"\nEncoder para {col}:")
    print("Clases:", encoder.classes_)



Encoder para sexo:
Clases: ['hombre' 'mujer']

Encoder para estado salud:
Clases: ['Excelente' 'Malo' 'Normal' 'Otro']

Encoder para ingresos:
Clases: ['Alto' 'Bajo' 'Medio']

Encoder para deporte:
Clases: ['No' 'Sí']

Encoder para educacion:
Clases: ['Postgrado' 'Primaria' 'Secundaria' 'Universitaria']


In [21]:
# Paso 1: Identificar filas con valores nulos o NaN
df_nan = df_eda[df_eda.isnull().any(axis=1)]

# Paso 2: Mostrar las filas que contienen valores nulos o NaN
display(df_nan)

# Paso 3: Contar la cantidad de filas con valores nulos o NaN
print(f"Total de filas con valores nulos o NaN: {len(df_nan)}")

,sexo,edad,estado salud,ingresos,deporte,educacion
4,NaN,30,NaN,Medio,Sí,Secundaria
9,mujer,43,Malo,Medio,NaN,Primaria
10,hombre,46,NaN,Alto,Sí,Universitaria
12,hombre,51,Excelente,NaN,Sí,Primaria
15,mujer,58,Malo,Medio,No,NaN
17,mujer,63,NaN,Alto,No,Postgrado
24,hombre,25,Malo,NaN,Sí,Primaria
29,mujer,18,NaN,Bajo,Sí,NaN
34,hombre,15,NaN,Alto,No,Universitaria
38,hombre,76,NaN,Bajo,No,Universitaria


Total de filas con valores nulos o NaN: 15


Vemos que en la columna `sexo` aun tenemos un NaN, esto es debido a que en esa fila, en alguna otra columna, hay algun NaN y por lo tanto no se ha realizado una prediccion.

Hay que acordarse de que se ha utilizado para el entrenamiento un dataframe con solo filas sin NaN.

Ahora debemos decidir que hacer con esas fila con NaN de la columna sexo:
*   La borramos
*   Le asignamos manualmente un valor

Como tenemos 27 hombre y 26 mujer, vamos a asignar a este NaN el valor de mujer, asi queda completamente balanceado

In [27]:
# Identificar filas donde 'sexo' es NaN
filas_sexo_nan = df_eda['sexo'].isna()

# @title String fields
# @markdown ### Elija un GENERO para introducirlo manualmente:
GENERO = 'mujer' # @param ["mujer", "hombre"]

# Cambiar NaN a 'No especificado' en la columna 'sexo'
df_eda.loc[filas_sexo_nan, 'sexo'] = GENERO

print("\nDataFrame después de cambiar NaN en 'sexo':")
print(df_eda)


DataFrame después de cambiar NaN en 'sexo':
      sexo  edad estado salud ingresos deporte      educacion
0   hombre     9         Malo     Bajo      Sí     Secundaria
1    mujer    22       Normal    Medio      No       Primaria
2   hombre    25    Excelente     Alto      Sí  Universitaria
3    mujer    27       Normal     Bajo      No      Postgrado
4    mujer    30          NaN    Medio      Sí     Secundaria
5   hombre    32         Malo     Bajo      No       Primaria
6   hombre    35       Normal     Alto      Sí  Universitaria
7    mujer    37    Excelente    Medio      No     Secundaria
8   hombre    40       Normal     Bajo      Sí      Postgrado
9    mujer    43         Malo    Medio     NaN       Primaria
10  hombre    46          NaN     Alto      Sí  Universitaria
11   mujer    48         Malo     Bajo      No     Secundaria
12  hombre    51    Excelente      NaN      Sí       Primaria
13   mujer    54         Malo     Bajo      No      Postgrado
14  hombre    56       No

In [28]:
# Verificar si hay NaN en la columna 'sexo'
nan_en_sexo = df_eda['sexo'].isna().sum()

print(f"Número de valores NaN en la columna 'sexo': {nan_en_sexo}")

if nan_en_sexo == 0:
    print("No hay valores NaN en la columna 'sexo'.")
else:
    print("Hay valores NaN en la columna 'sexo'.")

    # Si hay NaN, mostrar las filas que los contienen
    filas_con_nan = df_eda[df_eda['sexo'].isna()]
    print("\nFilas con NaN en la columna 'sexo':")
    print(filas_con_nan)

# Mostrar la distribución de valores en la columna 'sexo'
print("\nDistribución de valores en la columna 'sexo':")
print(df_eda['sexo'].value_counts(dropna=False))


Número de valores NaN en la columna 'sexo': 0
No hay valores NaN en la columna 'sexo'.

Distribución de valores en la columna 'sexo':
sexo
hombre    27
mujer     27
Name: count, dtype: int64


In [29]:
df_eda.info()
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sexo          54 non-null     object
 1   edad          54 non-null     int64 
 2   estado salud  47 non-null     object
 3   ingresos      51 non-null     object
 4   deporte       51 non-null     object
 5   educacion     50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sexo          54 non-null     object
 1   edad          54 non-null     int64 
 2   estado salud  47 non-null     object
 3   ingresos      51 non-null     object
 4   deporte       51 non-null     object
 5   educacion     50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB
